In [1]:
import time, re, openpyxl
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# 코사인유사도 계산
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도
#tqbm
from tqdm.notebook import tqdm
import pickle

In [2]:
### 업무추진비 현황 크롤링 page = range(1,3+1), 19년 9월 ~ 22년 2월 + 파일 이름 가져오기(file_list)
    
driver = webdriver.Chrome('D:/AIB/AI_Section3_Project/chromedriver.exe')
file_list = []

for page in range(1,3+1):
    haeundae_url = f'https://www.haeundae.go.kr/edu/board/list.do?boardId=BBS_0000044&menuCd=DOM_000000201004000000&paging=ok&startPage={page}'
    driver.get(haeundae_url)
    time.sleep(2)
    upload_list = driver.find_elements_by_css_selector('#font_size > div.table.respond > table > tbody > tr > td.ellipsis.left > a')
    href_list =[]
    
    for text in upload_list:
        # time.sleep(1)
        href = text.get_attribute('href')
        href_list.append(href)
        
    for i in href_list:
        # time.sleep(1)
        driver.get(i)
        # driver.find_element_by_css_selector('#font_size > article > table > tbody > tr:nth-child(3) > td > ul > li > a:nth-child(3)').click() #첨부파일 다운로드
        file = driver.find_element_by_css_selector('#font_size > article > table > tbody > tr:nth-child(3) > td > ul > li > a:nth-child(1)') #파일이름 리스트 가져오기
        file = file.text
        file_list.append(file)
        # print(file_list) #파일이름 리스트 가져오는 거 확인하기

C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\2384787763.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:/AIB/AI_Section3_Project/chromedriver.exe')
C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\2384787763.py:9: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  upload_list = driver.find_elements_by_css_selector('#font_size > div.table.respond > table > tbody > tr > td.ellipsis.left > a')
C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\2384787763.py:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  file = driver.find_element_by_css_selector('#font_size > article > table > tbody > tr:nth-child(3) > td > ul > li > a:nth-child(1)') #파일이름 리스트 가져오기


In [3]:
#파일 바로 가져올 수 있도록 리스트 내용물 수정하기 
file_name = [] 
for i in file_list:
    file_name.append(i.split(' ')[0])
print(file_name)
len(file_name)


['2022년2월구청장업무추진비집행현황.xlsx', '2022년1월구청장업무추진비집행현황.xlsx', '2021년12월구청장업무추진비집행현황.xlsx', '2021년11월구청장업무추진비집행현황.xlsx', '2021년10월구청장업무추진비집행현황.xlsx', '2021년9월구청장업무추진비집행현황.xlsx', '2021년8월구청장업무추진비집행현황.xlsx', '2021년7월구청장업무추진비집행현황.xlsx', '2021년6월구청장업무추진비집행현황.xlsx', '2021년5월구청장업무추진비집행현황.xlsx', '2021년4월구청장업무추진비집행현황.xlsx', '2021년3월구청장업무추진비집행현황.xlsx', '2021년2월구청장업무추진비집행현황.xlsx', '2021년1월구청장업무추진비집행현황.xlsx', '2020년12월구청장업무추진비집행현황.xlsx', '2020년11월구청장업무추진비집행현황.xlsx', '2020년10월구청장업무추진비집행현황.xlsx', '2020년9월구청장업무추진비집행현황.xlsx', '2020년8월구청장업무추진비집행현황.xlsx', '2020년7월구청장업무추진비집행현황.xlsx', '2020년6월구청장업무추진비집행현황.xlsx', '2020년5월구청장업무추진비집행현황.xlsx', '2020년4월구청장업무추진비집행현황.xlsx', '2020년3월구청장업무추진비집행현황.xlsx', '2020년2월구청장업무추진비집행현황.xlsx', '2020년1월구청장업무주진비집행현황.xlsx', '2019년12월구청장업무주진비집행현황.xlsx', '2019년11월구청장업무주진비집행현황.xlsx', '2019년10월구청장업무주진비집행현황.xlsx', '2019년9월구청장업무주진비집행현황.xlsx']


30

In [4]:
# csv 파일 불러오기 19년 9월 ~ 22년 2월
df = pd.DataFrame()
for file in file_name:
    basicpath = 'D:/AIB/AI_Section3_Project/crawling/'
    data = pd.read_excel(basicpath + file)
    print(file) #무슨 파일의 내용인지 확인 
    print(data) #data 풀어보기 
    df = pd.concat([df,data])
    
# print(data)
# print(filename[0:6])
# test = pd.read_excel('D:/AIB/AI_Section3_Project/crawling/' + filename[0])   
# print('D:/AIB/AI_Section3_Project/crawling/' + filename[0])
# test

2022년2월구청장업무추진비집행현황.xlsx
         사용일시          사용장소      사용액                                사용목적  \
0  2022-02-03   텐퍼센트 해운대구청점    39000                구정 주요 현안사항 논의 간담회 개최   
1  2022-02-03          초원복국   221000       구정 현안업무 소통 및 격려 오찬 간담회 개최(2차)   
2  2022-02-03        거대숯불구이   180000             구정 주요사업 홍보를 위한 기자간담회 개최   
3  2022-02-04          일품한우   240000        제3기 5급승진리더과정 교육대상자 격려 간담회 개최   
4  2022-02-04          일품한우   180000             구정 주요사업 홍보를 위한 기자간담회 개최   
5  2022-02-07           스시림   132000          좌2동 행정복지센터 직원 노고 격려 간담회 개최   
6  2022-02-08        명품포항물회   167000        신청사 설계공모 운영위원회 개최에 따른 간담회 개최   
7  2022-02-08         커피아르케   140000                          내방객용 음료 구입   
8  2022-02-09          레드도어   150000          좌3동 행정복지센터 직원 노고 격려 간담회 개최   
9  2022-02-09          세이로정   180000  반여1지구 풍수해 생활권 종합정비사업 출장에 따른 간담회 개최   
10 2022-02-09          한우리가   180000            도시재생활성화계획 수립용역 검토 간담회 개최   
11 2022-02-09           NaN    50000                          직

C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\1691977668.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data = pd.read_excel(basicpath + file)


2021년7월구청장업무추진비집행현황.xlsx
         사용일시         사용장소      사용액                               사용목적  \
0  2021-07-01         한우리가   240000   민선7기 4주년 맞이 간부 공무원 소통 간담회 개최(1차)   
1  2021-07-01         한우리가   240000   민선7기 4주년 맞이 간부 공무원 소통 간담회 개최(2차)   
2  2021-07-01    부산중동점 삿뽀로   196000          청렴구민감사관 운영 관련사항 검토 간담회 개최   
3  2021-07-02      해운대 대동일   225000           이건희 미술관 유치 추진에 따른 간담회 개최   
4  2021-07-02       일광수산횟집   240000         해운대형 플리마켓 운영에 따른 격려 간담회 개최   
5  2021-07-03          이레옥   200000         원더풀 미래대학 개강식 개최에 따른 간담회 개최   
6  2021-07-05    해운대소문난삼계탕   193000          구청사관리 현장근로자 노고격려 오찬간담회 개최   
7  2021-07-05        도미노피자   216300           구정 현안업무 추진에 따른 노고 격려품 구입   
8  2021-07-05     뉴욕핫도그앤커피   769500         코로나19 확진자 대응 등 비상근무 격려품 구입   
9  2021-07-05          NaN    50000                         직원 경조사비 지급   
10 2021-07-06        밍주센텀점   240000       주민복지국 현안사항 논의를 위한 관계자 간담회 개최   
11 2021-07-06          할매집   240000        대한민국 도시대상 평가 관련사항 검토 간담회 개최   
12 2021-07-06

In [ ]:
#df생성하고 필요없는 컬럼 삭제 
pd.set_option('display.max_columns', None, 'display.max_rows', None)
delete_columns = ['사용일시', '사용액', '사용목적', '사용대상', '사용자', '사용방법', 'r']
df1 = df.drop(delete_columns, axis='columns')
df1

In [35]:
#df확인하고 excel로 내보내기 
# type(df1)
# type(df)
# df
# print(df1.shape, df.shape)
# df1.reset_index()
df1.to_excel("df1.xlsx")

In [6]:
#excel에서 수정한 후 다시 가져오기 
#수정내역: 외1제거, 사용처/사용장소 하나로 합치기 
data = pd.read_excel(basicpath + 'df1.xlsx')

In [7]:
pd.set_option('display.max_columns', None, 'display.max_rows', 30)
haundae = data.dropna(axis=0)
haundae.isnull().sum() #결측치 없음 
len(haundae)

1057

In [8]:
result = haundae['사용장소'].str.split(',')
result = result.apply(lambda x: pd.Series(x))
result = result.stack().reset_index(level=1, drop=True).to_frame('restaurant')
# type(result)
result = result.reset_index()
type(result)
result.info()
result

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       1100 non-null   int64 
 1   restaurant  1100 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.3+ KB


,index,restaurant
0,0,텐퍼센트 해운대구청점
1,1,초원복국
2,2,거대숯불구이
3,3,일품한우
4,4,일품한우
...,...,...
1095,1149,해운대흑염소국밥
1096,1150,금수복국 본점
1097,1151,해운대이름난암소갈비
1098,1152,오르(or)


In [9]:
pd.set_option('display.max_columns', None, 'display.max_rows', 30)

idx = result[result['restaurant'] == '직원'].index
idx2 = result[result['restaurant'] == '경조사비'].index 
idx3 = result[result['restaurant'] == '직원 경조사비'].index 

result = result.drop(idx)
result = result.drop(idx2)
result = result.drop(idx3)

In [10]:
result.reset_index()
delete_columns = ['index']
result = result.drop(delete_columns, axis='columns')

In [11]:
result.info()
type(result)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044 entries, 0 to 1099
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   restaurant  1044 non-null   object
dtypes: object(1)
memory usage: 16.3+ KB


pandas.core.frame.DataFrame

In [12]:
#Official_store txt 파일 가져오기 
with open('D:/AIB/AI_Section3_Project/official_store.txt', 'r') as f:
    official = f.readlines()

len(official)

153977

In [13]:
odf = pd.read_csv('D:/AIB/AI_Section3_Project/official_store.txt', sep = '|')
odf.drop('last_load_dttm', axis=1, inplace=True)
odf.columns = ['상가업소번호', '상호명','지점명','상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드', '상권업종중분류명', '상권업종소분류코드',
	'상권업종소분류명',	'표준산업분류코드',	'표준산업분류명','시도코드', 	'시도명',	'시군구코드',	'시군구명',	'행정동코드',	'행정동명',	'법정동코드',
    '법정동명',	'지번코드',	'대지구분코드',	'대지구분명',	'지번본번지',	'지번부번지',	'지번주소',	'도로명코드',	'도로명',	'건물본번지',
    '건물부번지',	'건물관리번호',	'건물명',	'도로명주소',	'구우편번호',	'신우편번호',	'동정보',	'층정보',	'호정보',	'경도',	 '위도']

odf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153976 entries, 0 to 153975
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     153976 non-null  int64  
 1   상호명        153975 non-null  object 
 2   지점명        19193 non-null   object 
 3   상권업종대분류코드  153976 non-null  object 
 4   상권업종대분류명   153976 non-null  object 
 5   상권업종중분류코드  153976 non-null  object 
 6   상권업종중분류명   153976 non-null  object 
 7   상권업종소분류코드  153976 non-null  object 
 8   상권업종소분류명   153976 non-null  object 
 9   표준산업분류코드   144359 non-null  object 
 10  표준산업분류명    144359 non-null  object 
 11  시도코드       153976 non-null  int64  
 12  시도명        153976 non-null  object 
 13  시군구코드      153976 non-null  int64  
 14  시군구명       153976 non-null  object 
 15  행정동코드      153976 non-null  int64  
 16  행정동명       153875 non-null  object 
 17  법정동코드      153976 non-null  int64  
 18  법정동명       153976 non-null  object 
 19  지번코드       153976 non-n

In [14]:
# 음식점 데이터
odf = odf.loc[odf['상권업종대분류명'] == '음식']  
odf = odf.loc[(odf['시군구명'] == '해운대구')]

# 필요한 컬럼만
odf = odf[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '법정동명', '위도', '경도']]

# 좌동 | 중동 | 우동
odf = odf.loc[(odf['법정동명'] == '중동')]


In [15]:
odf

,상호명,상권업종중분류명,상권업종소분류명,표준산업분류명,법정동명,위도,경도
313,하드록카페부산,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.161502,129.165032
527,생어거스틴부산일루아점,별식/퓨전요리,동남아음식전문점,기타 외국식 음식점업,중동,35.158941,129.174511
630,수라원,제과제빵떡케익,제과점,제과점업,중동,35.166040,129.167401
662,스몰,양식,정통양식/경양식,서양식 음식점업,중동,35.162084,129.164546
3575,북경루,중식,중국음식/중국집,중식 음식점업,중동,35.171373,129.172461
...,...,...,...,...,...,...,...
153079,어썸버거,한식,한식/백반/한정식,한식 음식점업,중동,35.160856,129.177799
153279,이리온,한식,한식/백반/한정식,한식 음식점업,중동,35.167896,129.170271
153365,카페S,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.160800,129.192714
153511,바아방믈,유흥주점,빠/카페/스탠드빠,일반유흥 주점업,중동,35.160988,129.166912


In [16]:
# 칼럼명 단순화

odf.columns = ['name',  # 상호명
              'cate_1',  # 상권업종중분류명  
              'cate_2',  # 상권업종소분류명  
              'cate_3',  # 표준산업분류명   
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]

In [17]:
odf['cate_mix'] = odf['cate_1'] + odf['cate_2'] +  odf['cate_3']
odf['cate_mix'] = odf['cate_mix'].str.replace("/", " ")

In [18]:
odf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 313 to 153834
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      1022 non-null   object 
 1   cate_1    1022 non-null   object 
 2   cate_2    1022 non-null   object 
 3   cate_3    1018 non-null   object 
 4   dong      1022 non-null   object 
 5   lon       1022 non-null   float64
 6   lat       1022 non-null   float64
 7   cate_mix  1018 non-null   object 
dtypes: float64(2), object(6)
memory usage: 71.9+ KB


In [19]:
# cate_3가 null인 값 찾기 
for i in odf['cate_3'][odf['cate_3'].isnull()].index:
    print(odf.loc[i])

name          꽃보다돈까스뷔페
cate_1              부페
cate_2            종합부페
cate_3             NaN
dong                중동
lon          35.162902
lat         129.162823
cate_mix           NaN
Name: 33860, dtype: object
name            온더플레이트
cate_1              부페
cate_2            종합부페
cate_3             NaN
dong                중동
lon          35.160237
lat         129.165299
cate_mix           NaN
Name: 113377, dtype: object
name             오션테라스
cate_1              부페
cate_2            종합부페
cate_3             NaN
dong                중동
lon          35.158834
lat         129.170635
cate_mix           NaN
Name: 144725, dtype: object
name           비스트로314
cate_1              부페
cate_2            종합부페
cate_3             NaN
dong                중동
lon          35.161648
lat         129.164869
cate_mix           NaN
Name: 145430, dtype: object


In [20]:
odf = odf.fillna('종합부페')
odf.isnull().sum()

name        0
cate_1      0
cate_2      0
cate_3      0
dong        0
lon         0
lat         0
cate_mix    0
dtype: int64

In [11]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(odf['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

NameError: name 'odf' is not defined

In [22]:
odf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 313 to 153834
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      1022 non-null   object 
 1   cate_1    1022 non-null   object 
 2   cate_2    1022 non-null   object 
 3   cate_3    1022 non-null   object 
 4   dong      1022 non-null   object 
 5   lon       1022 non-null   float64
 6   lat       1022 non-null   float64
 7   cate_mix  1022 non-null   object 
dtypes: float64(2), object(6)
memory usage: 104.1+ KB


In [23]:
len(['상가업소번호', '상호명','지점명','상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드', '상권업종중분류명', '상권업종소분류코드',
	'상권업종소분류명',	'표준산업분류코드',	'표준산업분류명','시도코드', 	'시도명',	'시군구코드',	'시군구명',	'행정동코드',	'행정동명',	'법정동코드',
    '법정동명',	'지번코드',	'대지구분코드',	'대지구분명',	'지번본번지',	'지번부번지',	'지번주소',	'도로명코드',	'도로명',	'건물본번지',
    '건물부번지',	',건물관리번호',	'건물명',	'도로명주소',	'구우편번호',	'신우편번호',	'동정보',	'층정보',	'호정보',	'경도',	 '위도'])
'19912201|싱싱커피&토스트| |Q|음식|Q07|패스트푸드|Q07A10|토스트전문|I56192|피자, 햄버거, 샌드위치 및 유사 음식점업|26|부산광역시|26530|사상구|2653062000|괘법동|2653010400|괘법동|2653010400205780000|1|대지|578||부산광역시 사상구 괘법동 578|265303139004|부산광역시 사상구 괘감로|37||2653010400105780000002037|산업용품유통상가|부산광역시 사상구 괘감로 37|617726|46977||1|26|128.980455279266|35.1597741758499|2021-01-07 12:03:03'

'19912201|싱싱커피&토스트| |Q|음식|Q07|패스트푸드|Q07A10|토스트전문|I56192|피자, 햄버거, 샌드위치 및 유사 음식점업|26|부산광역시|26530|사상구|2653062000|괘법동|2653010400|괘법동|2653010400205780000|1|대지|578||부산광역시 사상구 괘법동 578|265303139004|부산광역시 사상구 괘감로|37||2653010400105780000002037|산업용품유통상가|부산광역시 사상구 괘감로 37|617726|46977||1|26|128.980455279266|35.1597741758499|2021-01-07 12:03:03'

In [24]:
odf

,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix
313,하드록카페부산,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.161502,129.165032,커피점 카페커피전문점 카페 다방비알콜 음료점업
527,생어거스틴부산일루아점,별식/퓨전요리,동남아음식전문점,기타 외국식 음식점업,중동,35.158941,129.174511,별식 퓨전요리동남아음식전문점기타 외국식 음식점업
630,수라원,제과제빵떡케익,제과점,제과점업,중동,35.166040,129.167401,제과제빵떡케익제과점제과점업
662,스몰,양식,정통양식/경양식,서양식 음식점업,중동,35.162084,129.164546,양식정통양식 경양식서양식 음식점업
3575,북경루,중식,중국음식/중국집,중식 음식점업,중동,35.171373,129.172461,중식중국음식 중국집중식 음식점업
...,...,...,...,...,...,...,...,...
153079,어썸버거,한식,한식/백반/한정식,한식 음식점업,중동,35.160856,129.177799,한식한식 백반 한정식한식 음식점업
153279,이리온,한식,한식/백반/한정식,한식 음식점업,중동,35.167896,129.170271,한식한식 백반 한정식한식 음식점업
153365,카페S,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.160800,129.192714,커피점 카페커피전문점 카페 다방비알콜 음료점업
153511,바아방믈,유흥주점,빠/카페/스탠드빠,일반유흥 주점업,중동,35.160988,129.166912,유흥주점빠 카페 스탠드빠일반유흥 주점업


In [25]:
#######네이버 지도 크롤링
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [26]:
# 네이버 지도 검색창에 [해운대 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

odf['naver_keyword'] = odf['dong'] + "%20" + odf['name']  # "%20"는 띄어쓰기를 의미합니다.
odf['naver_map_url'] = ''

In [27]:
for i, keyword in enumerate(odf['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {odf.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        # time.sleep(1)
        odf.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                odf.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                # time.sleep(0.8)
            except Exception as e2:
                print(e2)
                odf.iloc[i,-1] = np.nan
                # time.sleep(0.8)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

odf['naver_map_url'] = "https://m.place.naver.com/restaurant/" + odf['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
odf = odf.loc[~odf['naver_map_url'].isnull()]

이번에 찾을 키워드 : 0 / 1021 행 중동%20하드록카페부산


C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\2808594541.py:8: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  odf.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')


이번에 찾을 키워드 : 1 / 1021 행 중동%20생어거스틴부산일루아점


C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\2808594541.py:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  odf.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')


Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x008A7413+2389011]
	Ordinal0 [0x00839F61+1941345]
	Ordinal0 [0x0072C658+837208]
	Ordinal0 [0x007591DD+1020381]
	Ordinal0 [0x0075949B+1021083]
	Ordinal0 [0x00786032+1204274]
	Ordinal0 [0x00774194+1130900]
	Ordinal0 [0x00784302+1196802]
	Ordinal0 [0x00773F66+1130342]
	Ordinal0 [0x0074E546+976198]
	Ordinal0 [0x0074F456+980054]
	GetHandleVerifier [0x00A59632+1727522]
	GetHandleVerifier [0x00B0BA4D+2457661]
	GetHandleVerifier [0x0093EB81+569713]
	GetHandleVerifier [0x0093DD76+566118]
	Ordinal0 [0x00840B2B+1968939]
	Ordinal0 [0x00845988+1989000]
	Ordinal0 [0x00845A75+1989237]
	Ordinal0 [0x0084ECB1+2026673]
	BaseThreadInitThunk [0x7574FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77587A7E+286]
	RtlGetAppCont

In [33]:
odf.to_excel("odf.xlsx")


In [ ]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

naver_map_type_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []

visitor_review_list = []
chromedriver = 'D:/AIB/AI_Section3_Project/chromedriver.exe' 

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

for i, url in enumerate(odf['naver_map_url']):

    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(2)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text

        # 방문자 리뷰 수 
        visitor_reviews = driver.find_element_by_css_selector('#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span:nth-child(2) > a > em').text

        # 블로그 리뷰 수
        # blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text
        blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span:nth-child(3) > a > em").text
        
        # 블로그 별점 점수
        # star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text
        star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div:nth-child(1) > div._3uUKd > div._37n49 > span._1Y6hi._1A8_M > em").text


        naver_map_type_list.append(naver_map_type)
        visitor_review_list.append(visitor_reviews)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)


    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        naver_map_type_list.append('null')
        visitor_review_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')
        


        
driver.quit()
sub_driver.quit()


odf['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
odf['visitor_review'] = visitor_review_list 
odf['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수
odf['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
odf['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수



In [39]:
odf['visitor_review'] = visitor_review_list 
odf['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수
odf

C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\1892582246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odf['visitor_review'] = visitor_review_list
C:\Users\Huiseop\AppData\Local\Temp\ipykernel_20084\1892582246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odf['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수


,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,visitor_review,naver_blog_review_qty
313,하드록카페부산,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.161502,129.165032,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20하드록카페부산,https://m.place.naver.com/restaurant/37230679,양식,4.31,54,275
662,스몰,양식,정통양식/경양식,서양식 음식점업,중동,35.162084,129.164546,양식정통양식 경양식서양식 음식점업,중동%20스몰,https://m.place.naver.com/restaurant/1907433093,null,null,null,null
3698,원산냉면,한식,냉면집,한식 음식점업,중동,35.163686,129.169442,한식냉면집한식 음식점업,중동%20원산냉면,https://m.place.naver.com/restaurant/11846674,냉면,4.15,151,58
6895,청풍장,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.162651,129.166486,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20청풍장,https://m.place.naver.com/restaurant/13439412,null,null,null,null
7522,엘시티,한식,한식/백반/한정식,한식 음식점업,중동,35.161557,129.165845,한식한식 백반 한정식한식 음식점업,중동%20엘시티,https://m.place.naver.com/restaurant/1237400492,레지던스,4.42,165,438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150489,텐퍼센트커피,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.162539,129.179875,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20텐퍼센트커피,https://m.place.naver.com/restaurant/1834707365,카페,4.41,74,2
150541,만부부대찌개,한식,부대찌개/섞어찌개,한식 음식점업,중동,35.162413,129.170544,한식부대찌개 섞어찌개한식 음식점업,중동%20만부부대찌개,https://m.place.naver.com/restaurant/1915381061,"찌개,전골",4.52,111,65
150684,풍원장고기정찬,한식,한식/백반/한정식,한식 음식점업,중동,35.159637,129.182567,한식한식 백반 한정식한식 음식점업,중동%20풍원장고기정찬,https://m.place.naver.com/restaurant/1246097097,한식,4.35,852,564
150875,브론즈웨인커피,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.162651,129.166486,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20브론즈웨인커피,https://m.place.naver.com/restaurant/1165911809,카페,4.63,421,376


In [29]:
odf.to_excel("odf2.xlsx")

NameError: name 'odf' is not defined

In [35]:

odf2 = pd.read_excel('D:/AIB/AI_Section3_Project/crawling/' + 'odf2.xlsx')

In [3]:
odf2 = pd.read_excel('D:/AIB/AI_Section3_Project/crawling/' + 'odf3.xlsx')

In [18]:
odf2['visitor_review'] = ((odf2['visitor_review'] - odf2['visitor_review'].mean())/odf2['visitor_review'].std())

In [21]:
odf2['visitor_review']

0      0.700370
1      1.548680
2      1.548680
3      1.396588
4      1.396588
         ...   
360    0.026442
361   -0.515061
362   -0.259387
363   -0.546528
364   -0.538661
Name: visitor_review, Length: 365, dtype: float64

In [22]:
odf2['naver_blog_review_qty'] = ((odf2['naver_blog_review_qty'] - odf2['naver_blog_review_qty'].mean())/odf2['naver_blog_review_qty'].std())

In [27]:
odf2['office_review'] = ((odf2['office_review'] - odf2['office_review'].mean())/odf2['office_review'].std())

In [23]:

print(odf2.shape)
odf2=odf2.dropna(axis=0)
print(odf2.shape)

(365, 17)
(365, 17)


In [28]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(odf2['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [14]:

# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
odf2[['naver_blog_review_qty', 'naver_star_point', 'visitor_review', 'office_review']] = odf2[['naver_blog_review_qty', 'naver_star_point', 'visitor_review', 'office_review']].apply(pd.to_numeric)

In [30]:

place_simi_co = (
                + place_simi_cate * 0.7 # 공식 1. 카테고리 유사도
                + np.repeat([odf2['naver_blog_review_qty'].values], len(odf2['naver_blog_review_qty']) , axis=0) * 0.001  # 공식 2. 블로그 리뷰가 얼마나 많이 올라왔는지
                + np.repeat([odf2['naver_star_point'].values], len(odf2['naver_star_point']) , axis=0) * 0.005           # 공식 3. 블로그 별점이 얼마나 높은지
                + np.repeat([odf2['visitor_review'].values], len(odf2['visitor_review']) , axis=0) * 0.001    # 공식 4. 방문자 리뷰가 얼마나 많이 됐는지
                + np.repeat([odf2['office_review'].values], len(odf2['office_review']) , axis=0) * 0.005 # 공식 5. 방문자 리뷰가 얼마나 많이 됐는지
                )



# 아래 place_simi_co_sorted_ind 는 그냥 바로 사용하면 됩니다.
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(odf2, sorted_ind, place_name, top_n=10):
    
    place_title = odf2[odf2['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return odf2.iloc[similar_indexes]


# 상도국수를 포함해 5개 업체를 뽑아봅시다.

find_simi_place(odf2, place_simi_co_sorted_ind, '커피나무', 5)



,Unnamed: 0.1,Unnamed: 0,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,visitor_review,naver_blog_review_qty,office_review
18,18,139647,엣지993까페,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.156429,129.175156,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20엣지993까페,https://m.place.naver.com/restaurant/475538812,"카페,디저트",4.36,0.650547,2.177946,-0.161005
323,323,99634,스타벅스해운대점,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.160177,129.162203,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20스타벅스해운대점,https://m.place.naver.com/restaurant/37797967,카페,4.48,2.389124,-0.352737,-0.161005
316,316,94531,에스카피에,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.160237,129.165299,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20에스카피에,https://m.place.naver.com/restaurant/13573588,뷔페,4.47,0.956043,1.085263,-0.161005
312,312,147114,에이치.어스,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.160838,129.193406,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20에이치.어스,https://m.place.naver.com/restaurant/1197479803,카페,4.47,0.556144,1.011266,-0.161005
241,241,130888,스타벅스,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,중동,35.165086,129.164457,커피점 카페커피전문점 카페 다방비알콜 음료점업,중동%20스타벅스,https://m.place.naver.com/restaurant/1651676104,카페,4.44,1.995781,-0.414401,-0.161005


In [14]:

df1 = pd.read_excel('D:/AIB/AI_Section3_Project/crawling/' + 'df1.xlsx')

In [32]:
with open('odf2.pkl', 'wb') as f:
    pickle.dump(odf2, f)

In [26]:

list = (df1.value_counts())

사용장소      
일품한우          53
한우리가          53
직원            44
동백섬횟집         34
해운대이름난암소갈비    33
              ..
사천시락국밥         1
삼촌밥차런치펍        1
삿뽀로            1
상생라멘           1
흑송, 쿠무다        1
Length: 340, dtype: int64

In [28]:
pd.set_option('display.max_columns', None, 'display.max_rows', 30)
list

사용장소                   
일품한우                       53
한우리가                       53
직원                         44
동백섬횟집                      34
해운대이름난암소갈비                 33
해운대 대동일                    29
금수복국 본점                    25
일광수산횟집                     25
섬진강                        23
초원복국                       21
도미노피자                      21
경복궁                        18
한우리 가                      15
해운대소문난삼계탕                  14
해운대 대 동일                   13
소반                         13
할매집                        12
청오디피케이(주)센텀점               11
예이제                        10
소문난 막창                     10
직원 경조사비                    10
금수복국                       10
명향                         10
홈플러스온라인센텀시티점                9
류화돈                         9
뉴욕핫도그앤커피                    8
신드롬 치킨                      8
남포삼계탕                       8
밍주                          8
거대숯불구이                      8
밍주센텀점                       7
긴자해운대마린시티점                  7
삿뽀로 부산중동점(주)엔타스 

In [55]:
korean = find_simi_place(odf2, place_simi_co_sorted_ind, '테라스카페', 20)
korean = korean['name'].tolist()
korean
# korean[0], korean[1]

['엣지993까페',
 '스타벅스해운대점',
 '에스카피에',
 '에이치.어스',
 '스타벅스',
 '스타벅스',
 '스타벅스',
 '스타벅스',
 '스타벅스',
 '스타벅스',
 '스타벅스',
 '테라스카페',
 '카페테라스',
 '블랙업커피',
 '블랙업커피',
 '브론즈웨인커피',
 '디아트커피',
 '카페디아트',
 '마루팥빙수',
 '지비디']

In [115]:
find_simi_place(odf2, place_simi_co_sorted_ind, '오페라하우스', 20)

,Unnamed: 0,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,visitor_review,naver_blog_review_qty,office_review
259,127976,달인막창,한식,곱창/양구이전문,한식 음식점업,중동,35.163690,129.162439,한식곱창 양구이전문한식 음식점업,중동%20달인막창,https://m.place.naver.com/restaurant/1088169857,"곱창,막창,양",4.82,6492.0,464.0,0
44,57407,맥도날드,패스트푸드,패스트푸드,그외 기타 음식점업,중동,35.161842,129.170931,패스트푸드패스트푸드그외 기타 음식점업,중동%20맥도날드,https://m.place.naver.com/restaurant/11807421,햄버거,4.34,6504.0,117.0,0
126,96727,해운대소문난암소갈비,한식,갈비/삼겹살,한식 음식점업,중동,35.163247,129.166297,한식갈비 삼겹살한식 음식점업,중동%20해운대소문난암소갈비,https://m.place.naver.com/restaurant/11672261,"육류,고기요리",4.22,3794.0,2943.0,0
125,96539,해운대소문난암소갈비집,한식,갈비/삼겹살,한식 음식점업,중동,35.163247,129.166297,한식갈비 삼겹살한식 음식점업,중동%20해운대소문난암소갈비집,https://m.place.naver.com/restaurant/11672261,"육류,고기요리",4.22,3794.0,2943.0,0
122,99835,금수복국,한식,한식/백반/한정식,한식 음식점업,중동,35.162005,129.164238,한식한식 백반 한정식한식 음식점업,중동%20금수복국,https://m.place.naver.com/restaurant/11672174,복어요리,4.35,3383.0,2113.0,35
123,66713,금수식당,일식/수산물,복전문,한식 음식점업,중동,35.162435,129.164518,일식 수산물복전문한식 음식점업,중동%20금수식당,https://m.place.naver.com/restaurant/11672174,복어요리,4.35,3383.0,2113.0,0
124,77246,부산복국,한식,한식/백반/한정식,한식 음식점업,중동,35.161557,129.165845,한식한식 백반 한정식한식 음식점업,중동%20부산복국,https://m.place.naver.com/restaurant/11672174,복어요리,4.35,3383.0,2113.0,0
16,124011,스노잉클라우드,양식,정통양식/경양식,서양식 음식점업,중동,35.157284,129.181687,양식정통양식 경양식서양식 음식점업,중동%20스노잉클라우드,https://m.place.naver.com/restaurant/1010007916,"카페,디저트",4.74,2321.0,1091.0,0
1,54019,뉴욕버거,패스트푸드,패스트푸드,그외 기타 음식점업,중동,35.166040,129.167401,패스트푸드패스트푸드그외 기타 음식점업,중동%20뉴욕버거,https://m.place.naver.com/restaurant/33880194,햄버거,4.46,1648.0,1703.0,0
2,117800,버거인뉴욕,한식,한식/백반/한정식,한식 음식점업,중동,35.160946,129.164716,한식한식 백반 한정식한식 음식점업,중동%20버거인뉴욕,https://m.place.naver.com/restaurant/33880194,햄버거,4.46,1648.0,1703.0,0
